In [34]:
import h5py
from sklearn.model_selection import train_test_split
import numpy as np

In [35]:
data = h5py.File('data/Q1/MNIST_Subset.h5','r+')
X=data['X'][:]
Y=data['Y'][:]

In [36]:
X.shape,Y.shape

((14251, 28, 28), (14251,))

In [37]:
X = X.reshape(X.shape[0],-1)

In [38]:
X=X/255

In [39]:
np.unique(Y)

array([7, 9], dtype=uint8)

In [40]:
#as we only have 2 classes
Y = np.where(Y==7,0,1)

In [41]:
def one_hot_encoding(Y):
    n_classes = len(np.unique(Y))
    targets = Y.reshape(-1)
    return np.eye(n_classes)[targets],n_classes

In [42]:
Y,n_classes = one_hot_encoding(Y)

In [43]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2, random_state=42)

In [44]:
X_train.shape,X_test.shape,Y_train.shape,Y_test.shape

((11400, 784), (2851, 784), (11400, 2), (2851, 2))

In [45]:
#hyperparameters
epoch = 10
learning_rate = 0.01

In [46]:
def sigmoid(z,derative = False):
    if derative:
        return z*(1-z)
    return 1.0/(1.0+np.exp(-z))

In [56]:
def softmax(z,derative = False):
    if derative:
        return 
    return np.exp(z)/np.sum(np.exp(z),axis=0)

In [87]:
def compute_multiclass_loss(Y, Y_hat):
    L_sum = np.sum(np.multiply(Y, np.log(Y_hat)))
    m = Y.shape[1]
    return -(1/m) * L_sum

In [47]:
def init_weights(n_hidden_layers, number_of_neurons):
    weights_layers =[np.random.uniform(size=(X_train.shape[1],number_of_neurons[0]))]
    bias_layers = [np.random.uniform(size=(1,number_of_neurons[0]))]
    
    for i in range(1,n_hidden_layers):
        weights_layers.append(np.random.uniform(size=(weights_layers[i-1].shape[1],number_of_neurons[i])))
        bias_layers.append(np.random.uniform(size=(1,number_of_neurons[i])))
    
    weights_layers.append(np.random.uniform(size=(weights_layers[-1].shape[1],n_classes)))
    bias_layers.append(np.random.uniform(size=(1,n_classes)))
    
    return weights_layers,bias_layers

In [48]:
weight_layers,bias_layers = init_weights(2,[100,50])

In [49]:
for i,j in zip(weight_layers,bias_layers):
    print(i.shape,j.shape)

(784, 100) (1, 100)
(100, 50) (1, 50)
(50, 2) (1, 2)


In [83]:
def train_neural_network(X_train,Y_train,epoch,learning_rate):
    global weight_layers
    global bias_layers
    
    cost=[]
    
    #------epoch start here----------
    
    layer_outputs = []
    last_layer_output = X_train
    
    
    for weight,bias in zip(weight_layers[:-1],bais_layers[:-1]):
        layer_outputs.append(sigmoid(np.dot(last_layer_output,weight)+bias))
        last_layer_output=layer_outputs[-1]

    layer_outputs.append(softmax(np.dot(last_layer_output,weight_layers[-1])+bias_layers[-1]))
    cost.append(compute_multiclass_loss(Y_train,layer_outputs[-1]))
    print(cost)

In [88]:
train_neural_network(X_train,Y_train,epoch,learning_rate)

[53245.80121598119]
